# MediaPipe Hand

### To demonstrate Audio Control of the device using Hand Gesture control

In [1]:
import cv2
import mediapipe as mp

import numpy as np
import math

from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

static_image_mode = False
max_num_hands = 1
min_detection_confidence = 0.7
min_tracking_confidence = 0.7

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode, max_num_hands, min_detection_confidence, min_tracking_confidence)

cap = cv2.VideoCapture(0)

# Audio Variables
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
# volume.GetMute()
# volume.GetMasterVolumeLevel()
volRange = volume.GetVolumeRange()
minVol = volRange[0]
maxVol = volRange[1]
vol = 0
volBar = 400
volPer = 0

while cap.isOpened():
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    
    ldmark_list = []
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for id, ldmrk in enumerate(hand_landmarks.landmark):
                h, w, c = img.shape # -> h = 480, w = 640, c = 3
                cx, cy = int(ldmrk.x * w), int(ldmrk.y * h)
                ldmark_list.append([id, cx, cy])
                # cv2.circle(img, (cx, cy), 10, (226, 226, 226), cv2.FILLED)
                
            mp_drawing.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    
    if len(ldmark_list) != 0:
        x1, y1 = ldmark_list[4][1], ldmark_list[4][2]
        x2, y2 = ldmark_list[12][1], ldmark_list[12][2]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
 
        length = math.hypot(x2 - x1, y2 - y1)
 
        vol = np.interp(length, [30, 150], [minVol, maxVol])
        volBar = np.interp(length, [30, 150], [400, 150])
        volume.SetMasterVolumeLevel(vol, None)
        
    cv2.rectangle(img, (50, 150), (85, 400), (255, 0, 0), 3)
    cv2.rectangle(img, (50, int(volBar)), (85, 400), (255, 0, 0), cv2.FILLED)
    
    cv2.imshow('MediaPipe Hands', img)
    cv2.waitKey(1)
    if cv2.getWindowProperty('MediaPipe Hands', 4) < 1:
        break
        
cap.release()